# Detección de clases con aprendizaje supervisado reproducible

## Oier Mentxaka
https://cran.r-project.org/web/packages/xgboost/xgboost.pdf

### Carga de librerías


In [31]:
library(tidyverse)
library(caret)
library(xgboost)
source("Modelos/xgboost.R")
source("Preprocesamiento/utils.R")
source("Preprocesamiento/plots.R")

#### Asignación de la semilla

In [32]:
set.seed(123)

#### Lectura de fichero ya filtrado

In [33]:
data <- read.csv("../data/Statlog_XGB.csv", header = TRUE, sep = ",")

#### Separación de train y test y pesos

In [34]:
# Prepare the data for XGBoost
index <- createDataPartition(data$Class, p = 0.7, list = FALSE)

# Split the data
train <- data[index, ]
test <- data[-index, ]

### -1 porque el clasificador ha de empezar en 0 y en este dataset empieza en 1
y_train <- as.numeric(train$Class)-1
w_train <- as.numeric(train$Weights)
X_train <- as.matrix(train[, -which(names(train) %in% c("Class", "Weights"))])

y_test <- as.numeric(test$Class)-1
w_test <- as.numeric(test$Weights)
X_test <- as.matrix(test[, -which(names(test) %in% c("Class", "Weights"))])

#### Selección de variables 

Seleccionadas tras realizar un estudio con grid search

In [35]:
nrounds <- 10
objective <- "multi:softmax"

#### Creación del modelo

In [36]:
xgb_model <- xgboost(data = X_train, label = y_train, nrounds = nrounds, objective = objective, num_class = 2, weight = w_train)

[1]	train-mlogloss:0.568521 
[2]	train-mlogloss:0.495009 
[3]	train-mlogloss:0.438243 
[4]	train-mlogloss:0.399922 
[5]	train-mlogloss:0.370609 
[6]	train-mlogloss:0.337860 
[7]	train-mlogloss:0.316878 
[8]	train-mlogloss:0.297514 
[9]	train-mlogloss:0.269104 
[10]	train-mlogloss:0.254051 


#### Obtención de predicciones

In [37]:
predictions <- predict(xgb_model, newdata = X_test, weights = w_test)

#### Matriz de confusión

In [38]:
confusion_matrix <- confusion_matrix(predictions, y_test)

[1] "Confusion Matrix:"
           y_test
predictions   0   1
          0 184  50
          1  20  46


#### Precisión del modelo

In [39]:
# Calculate accuracy
accuracy <- sum(diag(confusion_matrix)) / sum(confusion_matrix)
print(paste("Accuracy:", round(accuracy, 2)))

[1] "Accuracy: 0.77"


In [ ]:
# Definir las opciones de la búsqueda de cuadrícula
objective_functions <- c("binary:logistic", "binary:logitraw", "binary:hinge") # "multi:softmax", "multi:softprob", "reg:linear", "reg:squarederror", "reg:gamma" Ejemplo de diferentes funciones objetivas
nrounds_values <- seq(10, 50, 10)  # Ejemplo de diferentes números de rondas de refuerzo

# Realizar la búsqueda de cuadrícula
best_params <- grid_search(X_train, y_train, X_test, y_test, objective_functions, nrounds_values)
print(paste("Best params:", best_params))